In [2]:
# Mount Google Drive to access files stored there
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/content/email.txt

In [3]:
# Install required packages from a requirements file located in Google Drive
!pip install -r '/content/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.5 MB/s eta 

In [4]:
# Install the LangChain Experimental package which includes advanced agents like Pandas agent
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.3 MB/s eta 0:00:00


In [5]:
# Import LangChain's prompt templating system for custom LLM instructions
from langchain.prompts import ChatPromptTemplate

# Import Gemini chat model and embedding model from langchain-google-genai integration
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Import a simple output parser to return LLM results as plain strings
from langchain_core.output_parsers import StrOutputParser

# Import Chroma as the vector store for semantic document retrieval
from langchain.vectorstores import Chroma

# Import CSVLoader to treat CSV rows as documents for RAG-style use
from langchain.document_loaders import CSVLoader

# Import Pandas DataFrame agent for structured data reasoning using code
from langchain_experimental.agents import create_pandas_dataframe_agent

# General-purpose libraries
import pandas as pd
import os

In [7]:
# Set your Google API key (for Gemini model access) as an environment variable
os.environ['GOOGLE_API_KEY'] = "your api key"

In [8]:
# Load your dataset (CSV) into a Pandas DataFrame
df = pd.read_csv("/content/Dataset.csv")


In [9]:
# Display the first few records to preview the dataset
df


,Date,Product Description,Invoice No,Qty,Unit,Rate,Total Amount,Discount,Outlet Name,Remarks
0,24-Jan-23,Dates Ajwa KSA JAMBO 500gm,AJSINV2023000003,7.0,Box,900.0,6300.0,630.0,A.J Super Outlet,NaN
1,24-Jan-23,Dates Anbara JAMBO KSA 4000gm,AJSINV2023000003,2.0,Box,4360.0,8720.0,872.0,A.J Super Outlet,NaN
2,24-Jan-23,Dates Anbara JAMBO KSA 500gm,AJSINV2023000003,20.0,Box,550.0,11000.0,1100.0,A.J Super Outlet,NaN
3,24-Jan-23,Dates Dabbas A UAE 1000gm,AJSINV2023000003,34.0,Box,350.0,11900.0,1190.0,A.J Super Outlet,NaN
4,24-Jan-23,Dates Dabbas A UAE 250gm,AJSINV2023000003,3.0,Box,100.0,300.0,30.0,A.J Super Outlet,NaN
...,...,...,...,...,...,...,...,...,...,...
177401,15-Apr-25,Dates Dabbas A UAE 500gm,MUTINV2025001692,1.0,Box,275.0,275.0,55.0,Uttara Outlet,NaN
177402,15-Apr-25,Dates Majdool A KSA 500gm,MUTINV2025001692,1.0,Box,650.0,650.0,130.0,Uttara Outlet,NaN
177403,16-Apr-25,Dates Rabia A UAE 1000gm,MUTINV2025001693,1.0,Box,690.0,690.0,138.0,Uttara Outlet,NaN
177404,16-Apr-25,Dates Safawi A KSA 1000gm,MUTINV2025001693,1.0,Box,1300.0,1300.0,260.0,Uttara Outlet,NaN


In [10]:
# Show metadata about the DataFrame: column names, types, non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177406 entries, 0 to 177405
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Date                 177406 non-null  object 
 1   Product Description  177406 non-null  object 
 2   Invoice No           177406 non-null  object 
 3   Qty                  177406 non-null  float64
 4   Unit                 177406 non-null  object 
 5   Rate                 177406 non-null  float64
 6   Total Amount         177406 non-null  float64
 7   Discount             177406 non-null  float64
 8   Outlet Name          177406 non-null  object 
 9   Remarks              268 non-null     object 
dtypes: float64(4), object(6)
memory usage: 13.5+ MB


In [11]:
# Load the same CSV into a document format for unstructured (RAG-style) queries
loader = CSVLoader(file_path="/content/.csv")
docs = loader.load()

In [12]:
# Display loaded documents (used for vector search + LLM)
docs

[Document(metadata={'source': '/content/drive/MyDrive/MSCSE Program/1st_Trisemester(spring_2025)/Decision Support System(CSE 6221)/Try code/Second Try/Dataset.csv', 'row': 0}, page_content='\ufeffDate: 24-Jan-23\nProduct Description: Dates Ajwa KSA JAMBO 500gm\nInvoice No: AJSINV2023000003\nQty: 7\nUnit: Box\nRate: 900\nTotal Amount: 6300\nDiscount: 630\nOutlet Name: A.J Super Outlet\nRemarks: '),
 Document(metadata={'source': '/content/drive/MyDrive/MSCSE Program/1st_Trisemester(spring_2025)/Decision Support System(CSE 6221)/Try code/Second Try/Dataset.csv', 'row': 1}, page_content='\ufeffDate: 24-Jan-23\nProduct Description: Dates Anbara JAMBO KSA 4000gm\nInvoice No: AJSINV2023000003\nQty: 2\nUnit: Box\nRate: 4360\nTotal Amount: 8720\nDiscount: 872\nOutlet Name: A.J Super Outlet\nRemarks: '),
 Document(metadata={'source': '/content/drive/MyDrive/MSCSE Program/1st_Trisemester(spring_2025)/Decision Support System(CSE 6221)/Try code/Second Try/Dataset.csv', 'row': 2}, page_content='\ufe

In [13]:
# Instantiate the embedding model (Google's Gemini Embeddings)
embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [14]:
# Instantiate Gemini chat model using the Flash-Lite version (faster, lightweight)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

In [15]:
persist_path = "/content/chroma_db"


In [ ]:
from langchain.vectorstores import Chroma

vector_store = Chroma.from_documents(
    docs,
    embedding=embedding_model,
    persist_directory=persist_path,
    collection_name="demo_00"
)


In [ ]:
# Configure the retriever to fetch top 20 most relevant documents from the vector DB
retriever = vector_store.as_retriever(search_kwargs={"k": 20})

In [ ]:
# Create a Pandas DataFrame agent that uses Gemini to generate and execute Python code
pandas_agent = create_pandas_dataframe_agent(
    llm,                     # The LLM for reasoning and code gen
    df,                      # The structured data
    verbose=True,            # Verbose logs showing internal steps (reasoning/code)
    allow_dangerous_code=True  # Allow code execution that can mutate state (⚠️ use carefully)
)

In [ ]:
# Define a prompt template for the RAG chain with explicit system and user roles
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a data assistant. Answer the question based on context only."),
    ("human", "{context}\n\nQuestion: {question}")
])

In [ ]:
# Set up a parser to convert LLM output into clean strings
output_parser = StrOutputParser()

In [ ]:
# Compose the RAG chain: Retrieve docs → Build prompt → Query Gemini → Parse result
rag_chain = (
    RunnableLambda(lambda query: {"docs": retriever.invoke(query), "question": query})  # Fetch relevant docs
    | RunnableLambda(lambda inputs: {  # Build prompt input
        "context": "\n\n".join([d.page_content for d in inputs["docs"]]),  # Combine doc contents
        "question": inputs["question"]
    })
    | prompt         # Format full prompt for the LLM
    | llm            # Query the Gemini model
    | output_parser  # Parse final output
)

In [ ]:
# Rule-based router function to detect query type and choose between structured (Pandas) or unstructured (RAG) handling
def route_query(query):
    # Keywords commonly found in structured queries
    structured_keywords = ["average", "sum", "total", "per", "max", "min", "group by", "week", "month"]
    if any(kw in query.lower() for kw in structured_keywords):
        return "pandas"  # Structured query → use Pandas agent
    else:
        return "rag"     # Unstructured query → use RAG pipeline

In [ ]:
# Unified interface to route and handle both query types
def hybrid_answer(query):
    route = route_query(query)
    if route == "pandas":
        return pandas_agent.invoke(query)   # Use Gemini + Python code on DataFrame
    else:
        return rag_chain.invoke(query)      # Use Gemini + document retrieval

In [ ]:
# Structured query example: numerical analysis using Pandas + Gemini
print(hybrid_answer("What is the average sales amount per branch?"))



> Entering new AgentExecutor chain...
Thought: I need to calculate the average total amount for each outlet. First, I need to group the dataframe by 'Outlet Name' and then calculate the mean of the 'Total Amount' for each group.
Action: python_repl_ast
Action Input:
```python
import pandas as pd
average_sales_per_branch = df.groupby('Outlet Name')['Total Amount'].mean()
print(average_sales_per_branch)
```Outlet Name
A.J Super Outlet       544.419354
Amtoli Outlet          510.422535
Badamtoli Outlet       505.000000
Badda Outlet           462.072770
Baipail Outlet         520.022912
Banani Outlet          697.043801
Banashree Outlet       572.432609
Gulshan-1 Outlet       770.706969
Keranigonj Outlet      494.165607
Mahamadpur Outlet      591.606802
Mirpur-11 Outlet       517.232216
Shahjahanpur Outlet    556.971496
Tejgaon Outlet         634.148475
Uttara Outlet          668.958472
Name: Total Amount, dtype: float64
I have calculated the average sales amount per branch using the gro

In [ ]:
# Unstructured query example: descriptive response via RAG
print(hybrid_answer("Describe the sales pattern"))

Here's a breakdown of the sales patterns based on the provided data:

**Product Types:**

*   **Dates Mabroom A KSA 1000gm:** This product appears to be a popular item, with sales occurring throughout the year.
*   **Dates Mabroom A KSA 500gm:** Another frequently sold item, with consistent sales.
*   **Dates Mabroom A KSA 250gm:** Sales for this size are less frequent compared to the 1000gm and 500gm options.

**Sales by Outlet:**

*   **Tejgaon Outlet:** This outlet has the most frequent sales, with a variety of product sizes sold.
*   **Baipail Outlet:** This outlet also has a good number of sales, primarily for the 500gm size.
*   **Keranigonj Outlet:** Sales are present, but less frequent.
*   **Uttara Outlet:** Sales are present, but less frequent.
*   **Gulshan-1 Outlet:** Only one sale recorded.
*   **Banashree Outlet:** Only one sale recorded.

**Pricing:**

*   The 1000gm size consistently sells for either 1600 or 1700.
*   The 500gm size sells for either 800 or 850.
*   The 

In [ ]:
# Another unstructured query: broad temporal sales analysis
print(hybrid_answer("Describe year base sales for all outlet"))

Here's a breakdown of sales by year and outlet based on the provided data:

**2023**

*   **Baipail Outlet:**
    *   February: 3150 (1000gm)
    *   March: 1350 (1000gm)
*   **Mahamadpur Outlet:**
    *   March: 3600 (1000gm) + 2700 (1000gm) + 1350 (1000gm) + 4050 (1000gm) + 2700 (1000gm)

**2024**

*   **Baipail Outlet:**
    *   February: 3450 (1000gm)
    *   March: 1380 (500gm) + 1035 (500gm)
*   **Total for Baipail Outlet:** 5865

**2025**

*   **Baipail Outlet:**
    *   March: 1380 (1000gm) + 6900 (1000gm) + 1380 (1000gm) + 690 (1000gm) + 4140 (1000gm) + 345 (500gm) + 690 (1000gm) + 690 (1000gm)
*   **Total for Baipail Outlet:** 15885
